In [3]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# The IR-assignment-2-data.txt should be copied into Local Directory, size is too large can't upload to Github
data = pd.read_csv('IR-assignment-2-data.txt', sep=" ", header=None)
rows = len(data.index)
db = {}   #Database for storing docID and Relevance Score

for i in range(0,rows):
    # Storing queries with qid:4 only
    if (data.at[i,1] == "qid:4"):
        db[i] = data.at[i,0]


def savefileMaxDcg(data, db):
    temp =data.drop((data.index[len(db):]))
    np.savetxt('query4max.txt', temp.values, fmt='%s', delimiter=" ")

savefileMaxDcg(data, db)

unsortedDb = db.items()
unsortedDb = list(unsortedDb) #List of tuple conatining docID and relevance pairs

# Sorting the database for getting maxDCG
db = sorted(db.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

In [ ]:
def findTotalFiles(db):
    count_3rel=0
    count_2rel=0
    count_1rel=0
    count_0rel=0
    for i in db:
        if (i[1] == 3):
            count_3rel+=1;
        elif (i[1] == 2):
            count_2rel+=1;
        elif (i[1] == 1):
            count_1rel+=1;
        else:
            count_0rel+=1;
        
    return math.factorial(count_3rel)*math.factorial(count_2rel)*math.factorial(count_1rel)*math.factorial(count_0rel)
#The total number of files will be 1!*17!*26!*59!
#findTotalFiles(db)

In [ ]:
def findDCG(data, length):
    ans = data[0][1];
    for i in range(1, length):
        temp = (data[i][1]/math.log2(i+1))
        ans= ans + temp    
    return ans   

In [ ]:
# MAx DCG using alternative formula is 28.98846753873482
def findAlternativeDCG(data):
    max_dcg_alt= 0
    i=1
    for val in db:
        max_dcg_alt += (pow(2, int(val[1]))-1)/np.log2(1+i)
        i += 1
    return max_dcg_alt
#print(findAlternativeDCG(db))

In [ ]:
# db contains sorted pairs
maxDcgValue = findDCG(db, len(db))
unsortedDcgValue = findDCG(unsortedDb, len(unsortedDb))

maxDcgValue_50 = findDCG(db, 51)
unsortedDcgValue_50 = findDCG(unsortedDb, 51)
#Printing nDCG
print("Max DCG is: {}".format(maxDcgValue))
print("nDCG at 50: {}".format(unsortedDcgValue_50/maxDcgValue_50))
print("nDCG whole Dataset: {}".format(unsortedDcgValue/maxDcgValue)) 

In [ ]:
def getRelevanceandTFpairs(db):
    for i in range(0,rows):
        if (data.at[i,1] == "qid:4"):
            s = float(data.at[i,76][3:])  #Storing the tf-idf value
            temp = (s,data.at[i,0])
            db[i] = (temp)
            
def drawGraph(recall, precision):
    plt.plot(recall, precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision - Recall Curve')
    plt.show()

def getPrecisionAndRecall(rel_tf_pairs):
    #ranking docs on the basis of the value of feature 45
    sortedByTF = sorted(rel_tf_pairs.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
    totalDocs = 103
    totalRelDocs = 44;
    retrivedRelDocs = 0;
    precision = []
    recall = []

    # Finding Recall and Precision 
    for i in range(0, len(sortedByTF)):
        if (sortedByTF[i][1][1] != 0):
            retrivedRelDocs+=1
        precision.append(retrivedRelDocs/(i+1))
        recall.append(retrivedRelDocs/totalRelDocs)
    drawGraph(recall, precision)


In [ ]:
rel_tf_pairs = {}    # Database for storing Relevance and tf-idf pairs
getRelevanceandTFpairs(rel_tf_pairs)
getPrecisionAndRecall(rel_tf_pairs)
